In [6]:
from azureml.core import Workspace, Dataset


subscription_id = '$ID'
resource_group = 'ML-resources'
workspace_name = 'ML-resources'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='CZ-FPZ_train_data')
dataset.to_pandas_dataframe()


workspace.write_config(path="/home/yossi/Desktop/new", file_name="config.json")

In [7]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
import pandas as pd

ws = Workspace.from_config()
experiment_name = 'test'

experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index= [''])
outputDf.T

/home/yossi/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  app.launch_new_instance()


,
Subscription ID,05ed07e3-df05-46ff-8687-8555449022bb
Workspace,ML-resources
Resource Group,ML-resources
Location,eastus
Experiment Name,test


In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "GPU-compute"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found exsist cluster, Use it ! ")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDART_D2_V2', max_nodes=6)

    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found exsist cluster, Use it ! 
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [12]:
from azureml.train.automl import AutoMLConfig
import logging


automl_settings = {
    "experiment_timeout_hours": 0.3,
    "enable_early_stopping": True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 1,
    "max_cores_per_iteration": 1,
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'classification',
                             primary_metric= 'AUC_weighted',
                             compute_target=compute_target,
                             blocked_models=['TensorFlowLinearClassifier'],
                             label_column_name = '0',
                             training_data = dataset,
                             n_cross_validations=2,
                             **automl_settings
                            )


In [13]:
remote_run = experiment.submit(automl_config, show_output = False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
test,AutoML_fc451b00-3f7a-4a05-be2b-af4a56e57153,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
from azureml.widgets import RunDetails
remote_run.wait_for_completion(show_output=True)


Experiment,Id,Type,Status,Details Page,Docs Page
test,AutoML_fc451b00-3f7a-4a05-be2b-af4a56e57153,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

In [14]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.32.0, current version:1.30.0
Package:azureml-core, training version:1.32.0, current version:1.30.0
Package:azureml-dataprep, training version:2.18.0, current version:2.15.1
Package:azureml-dataprep-native, training version:36.0.0, current version:33.0.0
Package:azureml-dataprep-rslex, training version:1.16.1, current version:1.13.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.30.0
Package:azureml-defaults, training version:1.32.0, current version:1.30.0
Package:azureml-interpret, training version:1.32.0, current version:1.30.0
Package:azureml-mlflow, training version:1.32.0, current version:1.30.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.30.0
Package:azureml-telemetry, training version:1.32.0, current version:1.30.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.30.0
Package:azureml-train-automl-runtime, training version:1.32.0, current ver

Run(Experiment: test,
Id: AutoML_5af815de-4388-45c1-acd5-43987761bd80_8,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0,
                                      class_weight=None, criterion='entropy',
                                      max_depth=None, max_features=0.3,
                                      max_leaf_nodes=None, max_samples=None,
                                      min_impurity_decrease=0.0,
                                      min_impurity_split=None,
                                      min_samples_leaf=0.01,
      

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show() 

In [ ]:
# Wait for the best model explanation run to complete
from azureml.interpret import ExplanationClient
from azureml.core.run import Run

model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=experiment, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

# Get the best run object
best_run, fitted_model = remote_run.get_output()

In [17]:
from azureml.interpret import ExplanationClient

client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=True)
exp_data = engineered_explanations.get_feature_importance_dict()
exp_data

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


{'1': 3.1195239237208483,
 '11': 0.9576921939077883,
 '16': 0.21080253396620363,
 '2': 0.14229072156330308,
 '12': 0.09280525789454484,
 '19': 0.05877687099890361,
 '20': 0.04340358977976776,
 '17': 0.04223609096530152,
 '44': 0.03829680068156537,
 '6': 0.03708060619787783,
 '18': 0.029133883902313192,
 '8': 0.022081078508902716,
 '22': 0.02033526241368925,
 '4': 0.018055669640736897,
 '551': 0.017028574070391943,
 '3': 0.016872382330707498,
 '7': 0.015304176145918585,
 '950': 0.010193609548646003,
 '30': 0.008570607486689565,
 '72': 0.008560623786477168,
 '558': 0.008334545251654838,
 '2811': 0.00761028751993115,
 '2152': 0.007182729511165007,
 '1271': 0.007166770262938112,
 '482': 0.00704476698871651,
 '41': 0.006945140196169216,
 '74': 0.0068718762685341115,
 '1503': 0.006847063111725042,
 '2978': 0.006218675847241782,
 '524': 0.006178231083747953,
 '1041': 0.0059061932669785405,
 '553': 0.0052652163880884135,
 '5': 0.005248466271490359,
 '2340': 0.005213078336445543,
 '1391': 0.005

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [24]:
import os
model_name = best_run.properties['model_name']

script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
model_dir = 'Model' # Local folder where the model will be stored temporarily
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)
    
best_run.download_file('outputs/model.pkl', model_dir + '/model.pkl')


In [23]:
description = 'AutoML Model trained on bank marketing data to predict if a client will subscribe to a term deposit'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id) # This will be written to the script file later in the notebook.
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')


AutoML5af815de48
